In [74]:
import numpy as np 
import pandas as pd 
df = pd.read_csv('./data/timeseries/weather.csv')
df.shape

(52696, 22)

In [8]:
import torch
from tqdm import *
train_len, val_len, test_len = 36600, 4982, 10253
data = torch.rand(train_len, 96, 21)
drop_out = 0.3

random_matrix = torch.ones(train_len, 96, 21)
# random_matrix = torch.zeros(self.data['x'], config.seq_len, enc_in)
for i in trange(train_len):
    mask = torch.rand(96, 21)
    mask[mask <= drop_out] = 0
    random_matrix[i] = mask
data = data * random_matrix
data.nonzero().shape, 36600 * 96 * 21

100%|██████████| 36600/36600 [00:00<00:00, 56640.01it/s]


(torch.Size([51648271, 3]), 73785600)

In [ ]:
import gzip
import pandas as pd

def read_ushcn_file(path: str) -> pd.DataFrame:
    records = []
    with gzip.open(path, 'rt') as f:
        for line in f:
            if not line.strip():
                continue
            station_id = line[0:6].strip()
            element = int(line[6:7])
            year = int(line[7:11])

            values, flags = {}, {}
            for i in range(13):  # 12个月 + 年值
                start_val = 12 + i * 7
                end_val = start_val + 5
                val_str = line[start_val:end_val].strip()
                flag_str = line[end_val:end_val+1].strip()

                val = None if val_str in ["", "-9999"] else int(val_str)
                flag = flag_str if flag_str else None

                if i < 12:
                    values[f"VALUE{i+1}"] = val
                    flags[f"FLAG{i+1}"] = flag
                else:
                    values["VALUE13"] = val
                    flags["FLAG13"] = flag

            records.append({
                "STATION_ID": station_id,
                "ELEMENT": element,
                "YEAR": year,
                **values,
                **flags
            })
    return pd.DataFrame(records)

# 示例
df = read_ushcn_file("./data/iregular/USHCN/ushcn_v2_monthly/9641C_201112_F52.pcp.gz")
df

In [ ]:
df.head()

In [ ]:
import torch 
data = torch.load('data/iregular/USHCN/ushcn.pt', map_location='cpu', weights_only=True)
len(data[0])

In [ ]:
data[0][0], data[0][1].shape, data[0][2].shape, data[0][3].shape

In [ ]:
for i in range(len(data)):
    print(f"Sample {i}:")
    print(f"  Time indices shape: {data[i][0]}")
    print(f"  Node indices shape: {data[i][1].shape}")
    # print(f"  Static features shape: {data[i][2].shape}")
    # print(f"  Mask shape: {data[i][3].shape}")   

In [ ]:
import argparse
parser = argparse.ArgumentParser('IMTS Forecasting')

parser.add_argument('--state', type=str, default='def')
parser.add_argument('-n',  type=int, default=int(1e8), help="Size of the dataset")
parser.add_argument('--hop', type=int, default=1, help="hops in GNN")
parser.add_argument('--nhead', type=int, default=1, help="heads in Transformer")
parser.add_argument('--tf_layer', type=int, default=1, help="# of layer in Transformer")
parser.add_argument('--nlayer', type=int, default=1, help="# of layer in TSmodel")
parser.add_argument('--epoch', type=int, default=1000, help="training epoches")
parser.add_argument('--patience', type=int, default=10, help="patience for early stop")
parser.add_argument('--history', type=int, default=24, help="number of hours (months for ushcn and ms for activity) as historical window")
parser.add_argument('-ps', '--patch_size', type=float, default=24, help="window size for a patch")
parser.add_argument('--stride', type=float, default=24, help="period stride for patch sliding")
parser.add_argument('--logmode', type=str, default="a", help='File mode of logging.')

parser.add_argument('--lr',  type=float, default=1e-3, help="Starting learning rate.")
parser.add_argument('--w_decay', type=float, default=0.0, help="weight decay.")
parser.add_argument('-b', '--batch_size', type=int, default=32)

parser.add_argument('--save', type=str, default='experiments/', help="Path for save checkpoints")
parser.add_argument('--load', type=str, default=None, help="ID of the experiment to load for evaluation. If None, run a new experiment.")
parser.add_argument('--seed', type=int, default=1, help="Random seed")
parser.add_argument('--dataset', type=str, default='physionet', help="Dataset to load. Available: physionet, mimic, ushcn")

# value 0 means using original time granularity, Value 1 means quantization by 1 hour, 
# value 0.1 means quantization by 0.1 hour = 6 min, value 0.016 means quantization by 0.016 hour = 1 min
parser.add_argument('--quantization', type=float, default=0.0, help="Quantization on the physionet dataset.")
parser.add_argument('--model', type=str, default='tPatchGNN', help="Model name")
parser.add_argument('--outlayer', type=str, default='Linear', help="Model name")
parser.add_argument('-hd', '--hid_dim', type=int, default=64, help="Number of units per hidden layer")
parser.add_argument('-td', '--te_dim', type=int, default=10, help="Number of units for time encoding")
parser.add_argument('-nd', '--node_dim', type=int, default=10, help="Number of units for node vectors")
parser.add_argument('--gpu', type=str, default='0', help='which gpu to use.')

args = parser.parse_args([])

In [ ]:
args.n_months = 48 # 48 monthes
args.pred_window = 1 # predict future one month

def USHCN_time_chunk(data, args, device):

	chunk_data = []

	for b, (record_id, tt, vals, mask) in enumerate(data):
		for st in range(0, args.n_months - args.history - args.pred_window + 1, args.pred_window):
			et = st + args.history + args.pred_window
			if(et == args.n_months):
				indices = torch.where((tt >= st) & (tt <= et))[0]
			else:
				indices = torch.where((tt >= st) & (tt < et))[0]
			t_bias = torch.tensor(st).to(device)
			chunk_data.append((record_id, tt[indices]-t_bias, vals[indices], mask[indices], t_bias))

	return chunk_data

In [ ]:
chunk_data = USHCN_time_chunk(data, args, device='cpu')

In [ ]:
len(chunk_data)

In [ ]:
len(chunk_data[0])

In [ ]:
chunk_data[0][1]

In [ ]:
chunk_data[2][1]

In [ ]:
chunk_data[5][2]

In [70]:
import pandas as pd 

df = pd.read_csv('data/iregular/USHCN/small_chunked_sporadic.csv')
df.keys()

Index(['ID', 'Time', 'Value_0', 'Value_1', 'Value_2', 'Value_3', 'Value_4',
       'Mask_0', 'Mask_1', 'Mask_2', 'Mask_3', 'Mask_4'],
      dtype='object')

In [73]:
df.groupby(level=0)